# Installation

In [1]:
%%writefile requirements.txt
unstructured
tiktoken
pinecone-client
pypdf
openai
langchain
pandas
numpy
python-dotenv
accelerate
transformers
langchain-huggingface

Writing requirements.txt


In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 24.1 MB/s eta 0:00:00
   ━

In [8]:
!pip install -U langchain-community
!pip install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.5
    Uninstalling aiohttp-3.10.5:
      Successfully uninstalled aiohttp-3.10.5


# Imports

In [9]:
import langchain
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import ServerlessSpec
# from langchain_pinecone import Pinecone
# from pinecone.grpc import PineconeGRPC as Pinecone
# from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

In [10]:
from google.colab import userdata
import os
os.environ['PINECONE_KEY'] = userdata.get('PINECONE_KEY')


SecretNotFoundError: Secret PINECONE_KEY does not exist.

In [ ]:

## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

doc = read_doc('documents/')
len(doc)

In [ ]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs


documents=chunk_data(docs=doc)
len(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
query_result = embeddings.embed_query("Hello world")
len(query_result)

In [ ]:
pc = Pinecone(api_key=os.environ['PINECONE_KEY'])

In [ ]:
index_name = "pdf-qa-index"
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.add_documents(doc)

In [ ]:
def retrieve_query(query,k=2):
    matching_results=vector_store.similarity_search(query,k=k)
    return matching_results

In [ ]:
results = vector_store.similarity_search(query="What is the main topic of this document?", k=2)

for res in results:
    print(f"* {res.page_content}")

In [ ]:
llm = HuggingFacePipeline(
    model_id="bigscience/T0_3B",
    model_kwargs={"temperature": 0.1, "max_length": 512}
)
chain = load_qa_chain(llm, chain_type="stuff")


In [ ]:
query = "What is the document about?"
doc_search=retrieve_query(query)
print(doc_search)
response=chain.run(input_documents=doc_search,question=query)
